### Load the data

In [71]:
# Useful starting lines
%matplotlib inline
import numpy as np
import datetime
import matplotlib.pyplot as plt

from helpers import *
from implementations import *
# from training_procedure import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
y, tx, ids = load_csv_data("train.csv", sub_sample=False)

# mean = []
# for i in range(tx.shape[1]):
#     tx[:, i][tx[:, i] == -999] = np.mean(tx[:, i][tx[:, i] != -999])
#     mean.append(np.mean(tx[:, i][tx[:, i] != -999]))

ind = np.where(tx[:, 22] > 1)
tx = tx[tx[:, 22] > 1]

# transform the categorical features into one-hot features
tx = np.hstack((tx, 
    (tx[:, 22] == 2).astype(np.float64)[:, np.newaxis],
    (tx[:, 22] == 3).astype(np.float64)[:, np.newaxis]))
tx = np.delete(tx, 22, axis = 1)

# standardization
tx_mean = np.mean(tx, axis=0)
tx_std = np.std(tx, axis=0)
tx[:, :-2] = (tx[:, :-2] - tx_mean[:-2]) / tx_std[:-2]

# feature augmentation
# quadratic terms
n = tx.shape[1]
for i in range(n):
    new_features = np.multiply(tx[:, i:n], tx[:, 0:n-i])
    tx = np.hstack([tx, new_features])

# drop all-zero columns, which may cause tx.T @ tx a singular matrix
zero_idx = np.argwhere(np.all(tx == 0, axis=0))
tx = np.delete(tx, zero_idx, axis=1)

# quartic terms # improvement ~1%, but too many features!
# new_features = np.multiply(tx[:, n:], tx[:, n:])
# nf_mean_1 = np.mean(new_features, axis=0)
# nf_std_1 = np.std(new_features, axis=0)
# new_features = (new_features - nf_mean_1) / nf_std_1
# tx = np.hstack([tx, new_features])

# # cubic terms # small improvement ~ 0.16%
# new_features = np.power(tx[:, :n-4], 3)
# nf_mean_2 = np.mean(new_features, axis=0)
# nf_std_2 = np.std(new_features, axis=0)
# new_features = (new_features - nf_mean_2) / nf_std_2
# tx = np.hstack([tx, new_features])

# sin, cos term # improvement ~ 4%
tx = np.hstack([tx, np.cos(tx),np.sin(2*tx),np.cos(2*tx)])
# cos_, _, _ = standardize(np.cos(tx))
# sin_, _, _ = standardize(np.sin(2*tx))
# cos__, _, _ = standardize(np.cos(2*tx))
# tx = np.hstack([tx, cos_, sin_, cos__])

# add the bias term to features
bias_term = np.ones([tx.shape[0], 1])
tx = np.hstack([tx, bias_term])

# drop duplicate columns
tx, unique_idx = np.unique(tx, axis=1, return_index=True)


In [73]:
tx.shape, y[ind].shape

((72543, 2097), (72543,))

In [6]:
# y, tx, ids = load_csv_data("train.csv", sub_sample=False)

# # change outlier values to the mean without them
# mean = []
# for i in range(tx.shape[1]):
#     tx[:, i][tx[:, i] == -999] = np.mean(tx[:, i][tx[:, i] != -999])
#     mean.append(np.mean(tx[:, i][tx[:, i] != -999]))
# # transform the categorical features into one-hot features
# #tx_ = np.delete(tx, 22, axis = 1)
# #tx = np.hstack((tx,tx_**2,tx_**3,np.sin(tx_),np.cos(tx_),np.sin(2*tx_),np.cos(2*tx_),np.sin(tx_)**2,np.cos(tx_)**2,np.sin(0.5*tx_),np.cos(0.5*tx_),
# #    (tx[:, 22] == 0).astype(np.float64)[:, np.newaxis],
# #    (tx[:, 22] == 1).astype(np.float64)[:, np.newaxis],
# #    (tx[:, 22] == 2).astype(np.float64)[:, np.newaxis],
# #    (tx[:, 22] == 3).astype(np.float64)[:, np.newaxis]))
# ind = np.where(tx[:, 22] > 1)
# tx_ = np.delete(tx[tx[:, 22] > 1], 22, axis = 1)
# tx__ = np.hstack([tx[tx[:, 22] > 1],tx_[:,0].reshape((tx_.shape[0],1))*tx_,tx_[:,1].reshape((tx_.shape[0],1))*tx_,tx_[:,2].reshape((tx_.shape[0],1))*tx_,tx_[:,3].reshape((tx_.shape[0],1))*tx_,tx_[:,4].reshape((tx_.shape[0],1))*tx_,tx_[:,5].reshape((tx_.shape[0],1))*tx_,tx_[:,6].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,7].reshape((tx_.shape[0],1))*tx_,np.sin(tx_),tx_[:,8].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,9].reshape((tx_.shape[0],1))*tx_,tx_[:,10].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,11].reshape((tx_.shape[0],1))*tx_,tx_[:,12].reshape((tx_.shape[0],1))*tx_,tx_[:,13].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,14].reshape((tx_.shape[0],1))*tx_,tx_[:,15].reshape((tx_.shape[0],1))*tx_,tx_[:,16].reshape((tx_.shape[0],1))*tx_,
#                 tx_[:,17].reshape((tx_.shape[0],1))*tx_,tx_[:,18].reshape((tx_.shape[0],1))*tx_,tx_[:,19].reshape((tx_.shape[0],1))*tx_,
#                 np.cos(tx_),np.sin(2*tx_),np.cos(2*tx_)])
# tx = np.hstack([tx_])
# tx = np.delete(tx, 22, axis = 1)

# # standardization
# tx_mean = np.mean(tx, axis=0)
# tx_std = np.std(tx, axis=0)
# tx = (tx - tx_mean) / tx_std

In [7]:
# #y.shape, tx.shape
# np.mean(tx, axis = 0).shape


(28,)

### Gradient Descent

In [74]:
# Define the parameters of the algorithm.
max_iters = 100
gamma = 0.0039

# Initialization
initial_w = np.zeros(tx.shape[1])

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= least_squares_GD(y[ind], tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

y_pred = tx @ gd_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Gradient descent accuracy:', np.mean(y_pred == y[ind]))

Gradient Descent: execution time = 4.883 seconds
Gradient Descent: loss = 0.29985534026201366
Gradient descent accuracy: 0.7971961457342541


In [ ]:
# cross validation for GD. Maybe No Need...

from helpers import *

k_fold = 5
seed = None
initial_w = np.zeros(tx.shape[1])
max_iters = 10 
gammas = np.logspace(-4, -2, 50)

# select gamma
gamma_ = hyperparameter_tuning_GD(tx, y[ind], k_fold, seed, initial_w, max_iters, gammas)

# Start gd
gd_ws, gd_losses = least_squares_GD(y[ind], tx, initial_w, max_iters, gamma_)

# Print result
print("Hyperparameter Selection: Gamma = {t}".format(t=gamma_))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

y_pred = tx @ gd_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Gradient descent accuracy:', np.mean(y_pred == y[ind]))

### Stochastic Gradient Descent

In [29]:
# Define the parameters of the algorithm.
max_iters = 10
gamma = 0.000000015
sgd_ws = np.zeros(tx.shape[1])

# Initialization
initial_w = sgd_ws

# Start gradient descent.
start_time = datetime.datetime.now()
sgd_ws, sgd_losses= least_squares_SGD(y[ind], tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Stochastic Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Stochastic Gradient Descent: loss = {t}".format(t=sgd_losses))

y_pred = tx @ sgd_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Stochastic gradient descent accuracy:', np.mean(y_pred == y[ind]))

Stochastic Gradient Descent: execution time = 39.388 seconds
Stochastic Gradient Descent: loss = 0.42943024203356345
Stochastic gradient descent accuracy: 0.6872199936589333


### Least Squares

In [75]:
# Start least squares.
start_time = datetime.datetime.now()
ls_ws, ls_losses= least_squares(y[ind], tx)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Least Squares: execution time = {t:.3f} seconds".format(t=exection_time))
print("Least Squares: loss = {t}".format(t=ls_losses))

y_pred = tx @ ls_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Least squares accuracy:', np.mean(y_pred == y[ind]))

Least Squares: execution time = 1.957 seconds
Least Squares: loss = 0.2414721647739247
Least squares accuracy: 0.8477316901699682


### Ridge Regression

In [76]:
# Start least squares.
start_time = datetime.datetime.now()
r_ws, r_losses = ridge_regression(y[ind], tx, 1e-17)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Ridge Regression: execution time = {t:.3f} seconds".format(t=exection_time))
print("Ridge Regression: loss = {t}".format(t=r_losses))

y_pred = tx @ r_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Ridge Regression accuracy:', np.mean(y_pred == y[ind]))


Ridge Regression: execution time = 2.103 seconds
Ridge Regression: loss = 0.241469934786494
Ridge Regression accuracy: 0.8477041203148478


In [ ]:
# # initialization
# degree = 1
# ratio = 0.8
# seed = None
# lambdas = np.logspace(-20, -15, 150)

# # select lambda
# lambda_ = hyperparameter_tuning_ridge_regression(tx, y[ind], degree, ratio, seed, lambdas)

# # Start ridge regression
# start_time = datetime.datetime.now()
# r_ws, r_losses = ridge_regression(y[ind], tx, lambda_)
# end_time = datetime.datetime.now()

# # Print result
# exection_time = (end_time - start_time).total_seconds()
# print("Ridge Regression: execution time = {t:.3f} seconds".format(t=exection_time))
# print("Hyperparameter Selection: Lambda = {t}".format(t=lambda_))
# print("Ridge Regression: loss = {t}".format(t=r_losses))

In [ ]:
# def build_k_indices(y, k_fold, seed):
#     """build k indices for k-fold.
#     return dimension: k-fold * interval
#     """
#     num_row = y.shape[0]
#     interval = int(num_row / k_fold)
#     np.random.seed(seed)
#     indices = np.random.permutation(num_row)
#     k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)] 
#     return np.array(k_indices)

In [ ]:
# # Cross Validation

# # Initialization
# seed = None
# k_fold = 5
# k_indices = build_k_indices(y[ind], k_fold, seed)

# train_acc = []
# val_acc = []
# for k in range(k_fold):
#     val_y = y[k_indices[k, :]]
#     val_x = tx[k_indices[k, :]]
#     train_idx = np.vstack([k_indices[:k, :], k_indices[k+1:, :]]).flatten()
#     train_y = y[train_idx]
#     train_x = tx[train_idx]

#     print('------', k, 'fold ------')
#     w, loss = ridge_regression(train_y, train_x, lambda_)
#     pred = train_x @ w
#     pred[pred > 0] = 1
#     pred[pred < 0] = -1
#     train_acc.append(np.mean(pred == train_y))
#     print('Train acc:', np.mean(pred == train_y))
#     pred = val_x @ w
#     pred[pred > 0] = 1
#     pred[pred < 0] = -1
#     val_acc.append(np.mean(pred == val_y))
#     print('Validation acc:', np.mean(pred == val_y))

# print('------ summary ------')
# print('Average Train acc:', sum(train_acc)/len(train_acc))
# print('Average Validation acc:', sum(val_acc)/len(val_acc))

## Logistic Regression

In [ ]:
# Define the parameters of the algorithm.
max_iters = 2
gamma = 0.0005


# Initialization
initial_w = np.zeros((tx.shape[1]))

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= logistic_regression(y[ind], tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

In [ ]:
def predict_labels_logit(w, tx):
    y_pred = sigmoid(np.dot(tx, w))
    y_pred[np.where(y_pred <= 0.5)] = -1
    y_pred[np.where(y_pred > 0.5)] = 1
    return y_pred
# predictions
y_pred = predict_labels_logit(gd_ws, tx)

# accuracy
(y_pred == y[ind]).mean()

## Regularized logistic regression

In [ ]:
# Define the parameters of the algorithm.
max_iters = 3
gamma = 0.0001
lambda_ = 0.00001

# Initialization
initial_w = np.zeros((tx.shape[1]))

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= reg_logistic_regression(y[ind], tx,lambda_, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

In [ ]:
# predictions
y_pred = predict_labels_logit(gd_ws, tx)

# accuracy
(y_pred == y[ind]).mean()

In [ ]:
#reg_logit_loss(y,tx,lambda_,gd_ws)
#(sigmoid(tx@gd_ws)<0).sum()
#tx@gd_ws

In [38]:

xx = build_poly(tx[:, 1:], 2)[:, 1:]
print(xx)
w_init = np.zeros(xx.shape[1])
train_first_feature(tx[:, 0], xx, w_init, 0.0001, 20, 0)

[[ 3.35898036e-01  4.36714239e-01 -1.08552886e+00 ...  3.79638208e-01
   1.85316930e+00  4.09065362e-01]
 [-7.16897628e-01 -4.66483417e-01  8.34344563e-02 ...  1.34748188e-02
   2.92526652e+00  1.11434713e-02]
 [-2.93845346e-01  6.69180568e-01 -5.16860831e-02 ...  4.94311036e-03
   2.31654982e+00  5.08748581e-04]
 ...
 [-3.90801233e-02  1.56653721e+01 -4.25443729e-01 ...  6.72410945e-02
   5.28207188e-02  3.90736156e-02]
 [-9.83018290e-01 -2.88263123e-01 -4.32164038e-01 ...  8.42839357e-02
   1.48404451e+00  1.14332379e+01]
 [ 2.11596036e-01 -3.80665862e-01  2.37009387e-01 ...  5.16061134e-04
   3.90509401e-01  5.70755550e-03]]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:32<00:00,  1.63s/it]


(array([-7.37578127e-03,  1.03218455e+00, -1.26879096e-01, -7.58821245e-03,
        -6.50856027e-03, -1.43952901e-03,  2.32947897e-02,  7.13720917e-03,
        -2.13964680e-02,  1.24781735e-02,  1.31138696e-01,  4.20883612e-03,
         8.45112644e-03, -4.43693947e-03, -1.83264673e-03, -9.61180968e-02,
         4.87404886e-03, -9.23575047e-04,  3.75859714e-01,  1.05601116e-03,
        -1.32052470e-02,  2.55997368e-03, -1.59968998e-03,  1.14561451e-02,
         2.66795043e-03,  3.49003764e-04, -2.12900829e-03, -2.32405599e-02,
        -3.23050165e-02, -3.69246963e-02, -1.61969676e-05,  9.43686552e-04,
        -2.88026539e-03,  3.71721876e-02, -2.05313668e-03,  3.01359501e-03,
        -6.94624991e-03, -1.26422817e-02,  4.32765941e-03,  9.13340245e-03,
         4.34263214e-03,  4.92183143e-03,  2.67928722e-02, -8.75852039e-03,
         4.02006423e-03, -1.82732932e-03,  4.01608429e-03,  4.43914916e-03,
         5.03880787e-03,  7.22943279e-03, -2.29663689e-03,  8.60163948e-03,
         4.7

### Prediction

#### To start, you shall upload text.csv to the branch

In [83]:
_, tx_test, ids_test = load_csv_data("test.csv", sub_sample=False)

# delete uninformative rows with '-999'
tx_test = tx_test[tx_test[:, 22] > 1]

# transform the categorical feature (PRI_jet_num) into onthot features
tx_test = np.hstack((tx_test, 
    (tx_test[:, 22] == 2).astype(np.float64)[:, np.newaxis],
    (tx_test[:, 22] == 3).astype(np.float64)[:, np.newaxis]))
tx_test = np.delete(tx_test, 22, axis = 1)

# standardization
tx_test_mean = np.mean(tx_test, axis=0)
tx_test_std = np.std(tx_test, axis=0)
tx_test[:, :-2] = (tx_test[:, :-2] - tx_test_mean[:-2]) / tx_test_std[:-2]

# feature augmentation
# quadratic terms
n = tx_test.shape[1]
for i in range(n):
    new_features = np.multiply(tx_test[:, i:n], tx_test[:, 0:n-i])
    tx_test = np.hstack([tx_test, new_features])

# drop all-zero columns, which may cause tX.T @ tX a singular matrix
zero_idx = np.argwhere(np.all(tx_test == 0, axis=0))
tx_test = np.delete(tx_test, zero_idx, axis=1)

# sin, cos term 
tx_test = np.hstack([tx_test, np.cos(tx_test),np.sin(2*tx_test),np.cos(2*tx_test)])

# add the bias term to features
bias_term = np.ones([tx_test.shape[0], 1])
tx_test = np.hstack([tx_test, bias_term])

# drop duplicate columns
tx_test = tx_test[:, unique_idx]

In [85]:
from helpers import *

OUTPUT_PATH = 'data/pred_GD.csv'
y_pred = predict(gd_ws, tx_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

OUTPUT_PATH = 'data/pred_ls.csv'
y_pred = predict(ls_ws, tx_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

OUTPUT_PATH = 'data/pred_ridge.csv'
y_pred = predict(r_ws, tx_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)